# Manhattan Crimes Clustering

## Import Libraries

In [28]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


## Connect to GDrive to read/write data

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


## Define Foursquare Credentials

In [ ]:


print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

## Other Definitions

In [4]:
DATA_LIMIT = 1000

## Fetch New York Crime Data

In [5]:
path = "/content/gdrive/MyDrive/Colab Notebooks/NYPD_Complaint_Data_Historic.csv"
nyc_complaint_data_hystoric = pd.read_csv(path)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
nyc_complaint_data_hystoric.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,JURIS_DESC,BORO_NM,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,101109527,12/31/2015,23:45:00,NaN,NaN,12/31/2015,113,FORGERY,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",COMPLETED,FELONY,N.Y. POLICE DEPT,BRONX,44.0,INSIDE,BAR/NIGHT CLUB,NaN,NaN,1007314.0,241257.0,40.828848,-73.916661,"(40.828848333, -73.916661142)"
1,153401121,12/31/2015,23:36:00,NaN,NaN,12/31/2015,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,COMPLETED,FELONY,N.Y. POLICE DEPT,QUEENS,103.0,OUTSIDE,NaN,NaN,NaN,1043991.0,193406.0,40.697338,-73.784557,"(40.697338138, -73.784556739)"
2,569369778,12/31/2015,23:30:00,NaN,NaN,12/31/2015,117,DANGEROUS DRUGS,503.0,"CONTROLLED SUBSTANCE,INTENT TO",COMPLETED,FELONY,N.Y. POLICE DEPT,MANHATTAN,28.0,NaN,OTHER,NaN,NaN,999463.0,231690.0,40.802607,-73.945052,"(40.802606608, -73.945051911)"
3,968417082,12/31/2015,23:30:00,NaN,NaN,12/31/2015,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,COMPLETED,MISDEMEANOR,N.Y. POLICE DEPT,QUEENS,105.0,INSIDE,RESIDENCE-HOUSE,NaN,NaN,1060183.0,177862.0,40.654549,-73.726339,"(40.654549444, -73.726338791)"
4,641637920,12/31/2015,23:25:00,12/31/2015,23:30:00,12/31/2015,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,COMPLETED,MISDEMEANOR,N.Y. POLICE DEPT,MANHATTAN,13.0,FRONT OF,OTHER,NaN,NaN,987606.0,208148.0,40.738002,-73.987891,"(40.7380024, -73.98789129)"


In [7]:
nyc_complaint_data_hystoric.shape

(1052656, 24)

Exclude rows with NaN on 'OFNS_DESC' column

In [8]:
nyc_complaint_data_hystoric = nyc_complaint_data_hystoric[nyc_complaint_data_hystoric['OFNS_DESC'].notna()]

Show crime categories

In [9]:
crimes = nyc_complaint_data_hystoric['OFNS_DESC'].unique()
pd.Series(crimes)

0                                  FORGERY
1          MURDER & NON-NEGL. MANSLAUGHTER
2                          DANGEROUS DRUGS
3             ASSAULT 3 & RELATED OFFENSES
4                           FELONY ASSAULT
5                        DANGEROUS WEAPONS
6                            PETIT LARCENY
7                            GRAND LARCENY
8                                  ROBBERY
9           OFFENSES AGAINST PUBLIC ADMINI
10          CRIMINAL MISCHIEF & RELATED OF
11                                    RAPE
12          INTOXICATED & IMPAIRED DRIVING
13                           HARRASSMENT 2
14                              SEX CRIMES
15                                BURGLARY
16                       CRIMINAL TRESPASS
17                 MISCELLANEOUS PENAL LAW
18                VEHICLE AND TRAFFIC LAWS
19           OFF. AGNST PUB ORD SENSBLTY &
20                                  FRAUDS
21          GRAND LARCENY OF MOTOR VEHICLE
22                OFFENSES INVOLVING FRAUD
23         

In [10]:
def getNearbyVenues(crimes, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for crime, lat, lng in zip(crimes, latitudes, longitudes):
        print(crime)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        response = requests.get(url).json()
        if ("groups" not in response["response"]):
          continue
        
        results = response["response"]['groups'][0]['items']
        if len(results) == 0:
          continue

        # return only relevant information for each nearby venue
        venues_list.append([(
            crime, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Crime', 
                  'Crime Latitude', 
                  'Crime Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Extract Manhattan crimes

In [11]:
manhattan_crimes = nyc_complaint_data_hystoric[ "MANHATTAN" == nyc_complaint_data_hystoric['BORO_NM'] ].reset_index(drop=True)

In [12]:
manhattan_crimes = manhattan_crimes[0:DATA_LIMIT]

In [13]:
manhattan_crimes.shape

(1000, 24)

In [ ]:
crimes_venues = getNearbyVenues(crimes=manhattan_crimes['OFNS_DESC'],
                                   latitudes=manhattan_crimes['Latitude'],
                                   longitudes=manhattan_crimes['Longitude']
                                  )

Streaming output truncated to the last 5000 lines.
DANGEROUS DRUGS
ASSAULT 3 & RELATED OFFENSES
DANGEROUS DRUGS
PETIT LARCENY
PETIT LARCENY
GRAND LARCENY
GRAND LARCENY
GRAND LARCENY
DANGEROUS DRUGS
HARRASSMENT 2
FELONY ASSAULT
CRIMINAL MISCHIEF & RELATED OF
CRIMINAL MISCHIEF & RELATED OF
ASSAULT 3 & RELATED OFFENSES
ASSAULT 3 & RELATED OFFENSES
MISCELLANEOUS PENAL LAW
ASSAULT 3 & RELATED OFFENSES
FRAUDS
HARRASSMENT 2
ASSAULT 3 & RELATED OFFENSES
PETIT LARCENY
PETIT LARCENY
PETIT LARCENY
ASSAULT 3 & RELATED OFFENSES
HARRASSMENT 2
HARRASSMENT 2
CRIMINAL MISCHIEF & RELATED OF
DANGEROUS DRUGS
PETIT LARCENY
HARRASSMENT 2
HARRASSMENT 2
FELONY ASSAULT
OFFENSES INVOLVING FRAUD
PETIT LARCENY
VEHICLE AND TRAFFIC LAWS
GRAND LARCENY
DANGEROUS DRUGS
HARRASSMENT 2
GRAND LARCENY
FELONY ASSAULT
HARRASSMENT 2
VEHICLE AND TRAFFIC LAWS
GRAND LARCENY
ROBBERY
DANGEROUS DRUGS
DANGEROUS DRUGS
DANGEROUS DRUGS
PETIT LARCENY
PETIT LARCENY
OFFENSES INVOLVING FRAUD
FRAUDULENT ACCOSTING
GRAND LARCENY
OTHER OFFENSE

ValueError: ignored

In [ ]:
crimes_venues.to_csv('crime_venues.csv', encoding='utf-8')

store to GDrive

In [ ]:
path = '/content/gdrive/My Drive/Colab Notebooks/crime_venues_{}.csv'.format(DATA_LIMIT)
crimes_venues.to_csv(path, encoding='utf-8')

In [15]:
crimes_venues = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/crime_venues.csv', index_col=0)

check the size of the resulting dataframe

In [16]:
print(crimes_venues.shape)
crimes_venues.head()

(21939, 7)


,Crime,Crime Latitude,Crime Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,FORGERY,40.828848,-73.916661,Bronx Museum of the Arts,40.831307,-73.919949,Art Museum
1,FORGERY,40.828848,-73.916661,Joyce Kilmer Park,40.828850,-73.922367,Park
2,FORGERY,40.828848,-73.916661,Hungry Bird,40.828820,-73.916828,Indian Restaurant
3,FORGERY,40.828848,-73.916661,Tiny's Pizza & Pasta,40.830930,-73.916626,Italian Restaurant
4,FORGERY,40.828848,-73.916661,Blimpie,40.826250,-73.921426,Sandwich Place


check how many venues were returned for each crime

In [17]:
crimes_venues.groupby('Crime').count()

,Crime Latitude,Crime Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Crime,,,,,,
ADMINISTRATIVE CODE,41,41,41,41,41,41
ARSON,178,178,178,178,178,178
ASSAULT 3 & RELATED OFFENSES,2866,2866,2866,2866,2866,2866
BURGLARY,703,703,703,703,703,703
CRIMINAL MISCHIEF & RELATED OF,2337,2337,2337,2337,2337,2337
CRIMINAL TRESPASS,73,73,73,73,73,73
DANGEROUS DRUGS,1439,1439,1439,1439,1439,1439
DANGEROUS WEAPONS,541,541,541,541,541,541
DISORDERLY CONDUCT,24,24,24,24,24,24


find out how many unique categories can be curated from all the returned venues

In [18]:
print('There are {} uniques categories.'.format(len(crimes_venues['Venue Category'].unique())))

There are 272 uniques categories.


save the resulting dataframe for later use, if needed

In [19]:
crimes_venues.head()

,Crime,Crime Latitude,Crime Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,FORGERY,40.828848,-73.916661,Bronx Museum of the Arts,40.831307,-73.919949,Art Museum
1,FORGERY,40.828848,-73.916661,Joyce Kilmer Park,40.828850,-73.922367,Park
2,FORGERY,40.828848,-73.916661,Hungry Bird,40.828820,-73.916828,Indian Restaurant
3,FORGERY,40.828848,-73.916661,Tiny's Pizza & Pasta,40.830930,-73.916626,Italian Restaurant
4,FORGERY,40.828848,-73.916661,Blimpie,40.826250,-73.921426,Sandwich Place


In [20]:
# one hot encoding
manhattan_onehot = pd.get_dummies(crimes_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Crime'] = crimes_venues['Crime'] 

# move crimes column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Crime,ATM,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bike Trail,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Campground,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Quad,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Exhibit,Eye Doctor,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Market,Martial Arts School,Mattress Store,Medical Center,Medical School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Post Office,Print Shop,Pub,Public Art,Puerto Rican Restaurant,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Road,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,State / Provincial Park,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tour Provider,Track,Track Stadium,Trail,Train,Train Station,Tram Station,Travel & Transport,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Weight Loss Center,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,FORGERY,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

examine new dataframe size

In [21]:
manhattan_onehot.shape

(21939, 273)

group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [22]:
manhattan_grouped = manhattan_onehot.groupby('Crime').mean().reset_index()
manhattan_grouped

,Crime,ATM,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bike Trail,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Campground,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Quad,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Exhibit,Eye Doctor,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Market,Martial Arts School,Mattress Store,Medical Center,Medical School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Post Office,Print Shop,Pub,Public Art,Puerto Rican Restaurant,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Road,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,State / Provincial Park,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tour Provider,Track,Track Stadium,Trail,Train,Train Station,Tram Station,Travel & Transport,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Weight Loss Center,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,ADMINISTRATIVE CODE,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390,0.000000,0

confirm new size

In [23]:
manhattan_grouped.shape

(29, 273)

print each crime along with the top 5 most common venues

In [24]:
num_top_venues = 5

for crime in manhattan_grouped['Crime']:
    print("----"+crime+"----")
    temp = manhattan_grouped[manhattan_grouped['Crime'] == crime].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ADMINISTRATIVE CODE----
                  venue  freq
0           Pizza Place  0.10
1  Caribbean Restaurant  0.10
2            Donut Shop  0.07
3              Pharmacy  0.07
4                  Bank  0.07


----ARSON----
                  venue  freq
0         Deli / Bodega  0.06
1            Donut Shop  0.05
2         Grocery Store  0.05
3  Fast Food Restaurant  0.04
4    Spanish Restaurant  0.04


----ASSAULT 3 & RELATED OFFENSES----
            venue  freq
0     Pizza Place  0.06
1  Sandwich Place  0.04
2      Donut Shop  0.04
3   Grocery Store  0.04
4   Deli / Bodega  0.04


----BURGLARY----
                 venue  freq
0          Pizza Place  0.06
1           Donut Shop  0.04
2        Deli / Bodega  0.04
3        Grocery Store  0.04
4  Fried Chicken Joint  0.03


----CRIMINAL MISCHIEF & RELATED OF----
                  venue  freq
0           Pizza Place  0.08
1         Deli / Bodega  0.04
2            Donut Shop  0.04
3  Fast Food Restaurant  0.03
4              Pharmacy  0.03

put that into a pandas dataframe

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Crime']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
crimes_venues_sorted = pd.DataFrame(columns=columns)
crimes_venues_sorted['Crime'] = manhattan_grouped['Crime']

for ind in np.arange(manhattan_grouped.shape[0]):
    crimes_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

crimes_venues_sorted.head()

,Crime,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ADMINISTRATIVE CODE,Caribbean Restaurant,Pizza Place,Pharmacy,Bank,Donut Shop,Bakery,Mobile Phone Shop,Fast Food Restaurant,Sandwich Place,Seafood Restaurant
1,ARSON,Deli / Bodega,Donut Shop,Grocery Store,Fast Food Restaurant,Discount Store,Pharmacy,Mobile Phone Shop,Spanish Restaurant,Park,Bus Station
2,ASSAULT 3 & RELATED OFFENSES,Pizza Place,Donut Shop,Grocery Store,Deli / Bodega,Sandwich Place,Mexican Restaurant,Pharmacy,Fast Food Restaurant,Fried Chicken Joint,Italian Restaurant
3,BURGLARY,Pizza Place,Donut Shop,Deli / Bodega,Grocery Store,Sandwich Place,Fast Food Restaurant,Mobile Phone Shop,Pharmacy,Fried Chicken Joint,Latin American Restaurant
4,CRIMINAL MISCHIEF & RELATED OF,Pizza Place,Deli / Bodega,Donut Shop,Mexican Restaurant,Italian Restaurant,Pharmacy,Sandwich Place,Grocery Store,Fast Food Restaurant,Chinese Restaurant


## Cluster Crimes

In [30]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Crime', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 3, 1, 1, 1, 1], dtype=int32)

In [ ]:
# add clustering labels
crimes_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [36]:
manhattan_merged = manhattan_crimes

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each crime
manhattan_merged = manhattan_merged.join(crimes_venues_sorted.set_index('Crime'), on='OFNS_DESC')

manhattan_merged.head() # check the last columns!

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,JURIS_DESC,BORO_NM,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,569369778,12/31/2015,23:30:00,NaN,NaN,12/31/2015,117,DANGEROUS DRUGS,503.0,"CONTROLLED SUBSTANCE,INTENT TO",COMPLETED,FELONY,N.Y. POLICE DEPT,MANHATTAN,28.0,NaN,OTHER,NaN,NaN,999463.0,231690.0,40.802607,-73.945052,"(40.802606608, -73.945051911)",1.0,Pizza Place,Donut Shop,Grocery Store,Mexican Restaurant,Deli / Bodega,Mobile Phone Shop,Sandwich Place,Spanish Restaurant,Fast Food Restaurant,Fried Chicken Joint
1,641637920,12/31/2015,23:25:00,12/31/2015,23:30:00,12/31/2015,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,COMPLETED,MISDEMEANOR,N.Y. POLICE DEPT,MANHATTAN,13.0,FRONT OF,OTHER,NaN,NaN,987606.0,208148.0,40.738002,-73.987891,"(40.7380024, -73.98789129)",1.0,Pizza Place,Donut Shop,Grocery Store,Deli / Bodega,Sandwich Place,Mexican Restaurant,Pharmacy,Fast Food Restaurant,Fried Chicken Joint,Italian Restaurant
2,608231454,12/31/2015,23:15:00,NaN,NaN,12/31/2015,235,DANGEROUS DRUGS,511.0,"CONTROLLED SUBSTANCE, POSSESSI",COMPLETED,MISDEMEANOR,N.Y. POLICE DEPT,MANHATTAN,7.0,OPPOSITE OF,STREET,NaN,NaN,987373.0,201662.0,40.720200,-73.988735,"(40.720199996, -73.988735082)",1.0,Pizza Place,Donut Shop,Grocery Store,Mexican Restaurant,Deli / Bodega,Mobile Phone Shop,Sandwich Place,Spanish Restaurant,Fast Food Restaurant,Fried Chicken Joint
3,415095955,12/31/2015,23:10:00,12/31/2015,23:10:00,12/31/2015,341,PETIT LARCENY,338.0,"LARCENY,PETIT FROM BUILDING,UN",COMPLETED,MISDEMEANOR,N.Y. POLICE DEPT,MANHATTAN,19.0,INSIDE,DRUG STORE,NaN,NaN,994327.0,218211.0,40.765618,-73.963623,"(40.765617688, -73.96362342)",1.0,Pizza Place,Donut Shop,Mexican Restaurant,Chinese Restaurant,Mobile Phone Shop,Deli / Bodega,Bank,Fast Food Restaurant,Grocery Store,Sandwich Place
4,178090167,12/31/2015,23:00:00,12/31/2015,23:05:00,12/31/2015,341,PETIT LARCENY,338.0,"LARCENY,PETIT FROM BUILDING,UN",COMPLETED,MISDEMEANOR,N.Y. POLICE DEPT,MANHATTAN,13.0,INSIDE,RESTAURANT/DINER,NaN,NaN,988113.0,206263.0,40.732828,-73.986063,"(40.732828332, -73.986062857)",1.0,Pizza Place,Donut Shop,Mexican Restaurant,Chinese Restaurant,Mobile Phone Shop,Deli / Bodega,Bank,Fast Food Restaurant,Grocery Store,Sandwich Place


## Examine Clusters

Cluster 1

In [63]:
cluster1 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[7] + list(range(24, manhattan_merged.shape[1]))]]
cluster1

,OFNS_DESC,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,FRAUDS,0.0,Fried Chicken Joint,Fast Food Restaurant,Pizza Place,Donut Shop,Supermarket,Park,Southern / Soul Food Restaurant,Pharmacy,Shopping Plaza,Breakfast Spot
164,FRAUDS,0.0,Fried Chicken Joint,Fast Food Restaurant,Pizza Place,Donut Shop,Supermarket,Park,Southern / Soul Food Restaurant,Pharmacy,Shopping Plaza,Breakfast Spot
178,FRAUDS,0.0,Fried Chicken Joint,Fast Food Restaurant,Pizza Place,Donut Shop,Supermarket,Park,Southern / Soul Food Restaurant,Pharmacy,Shopping Plaza,Breakfast Spot
187,FRAUDS,0.0,Fried Chicken Joint,Fast Food Restaurant,Pizza Place,Donut Shop,Supermarket,Park,Southern / Soul Food Restaurant,Pharmacy,Shopping Plaza,Breakfast Spot
196,FRAUDS,0.0,Fried Chicken Joint,Fast Food Restaurant,Pizza Place,Donut Shop,Supermarket,Park,Southern / Soul Food Restaurant,Pharmacy,Shopping Plaza,Breakfast Spot
438,FRAUDS,0.0,Fried Chicken Joint,Fast Food Restaurant,Pizza Place,Donut Shop,Supermarket,Park,Southern / Soul Food Restaurant,Pharmacy,Shopping Plaza,Breakfast Spot
564,FRAUDS,0.0,Fried Chicken Joint,Fast Food Restaurant,Pizza Place,Donut Shop,Supermarket,Park,Southern / Soul Food Restaurant,Pharmacy,Shopping Plaza,Breakfast Spot


In [65]:
pd.Series(cluster1['OFNS_DESC'].unique())

0    FRAUDS
dtype: object

Cluster 2

In [66]:
cluster2 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[7] + list(range(24, manhattan_merged.shape[1]))]]
cluster2

,OFNS_DESC,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,DANGEROUS DRUGS,1.0,Pizza Place,Donut Shop,Grocery Store,Mexican Restaurant,Deli / Bodega,Mobile Phone Shop,Sandwich Place,Spanish Restaurant,Fast Food Restaurant,Fried Chicken Joint
1,ASSAULT 3 & RELATED OFFENSES,1.0,Pizza Place,Donut Shop,Grocery Store,Deli / Bodega,Sandwich Place,Mexican Restaurant,Pharmacy,Fast Food Restaurant,Fried Chicken Joint,Italian Restaurant
2,DANGEROUS DRUGS,1.0,Pizza Place,Donut Shop,Grocery Store,Mexican Restaurant,Deli / Bodega,Mobile Phone Shop,Sandwich Place,Spanish Restaurant,Fast Food Restaurant,Fried Chicken Joint
3,PETIT LARCENY,1.0,Pizza Place,Donut Shop,Mexican Restaurant,Chinese Restaurant,Mobile Phone Shop,Deli / Bodega,Bank,Fast Food Restaurant,Grocery Store,Sandwich Place
4,PETIT LARCENY,1.0,Pizza Place,Donut Shop,Mexican Restaurant,Chinese Restaurant,Mobile Phone Shop,Deli / Bodega,Bank,Fast Food Restaurant,Grocery Store,Sandwich Place
5,GRAND LARCENY,1.0,Pizza Place,Fast Food Restaurant,Deli / Bodega,Donut Shop,Sandwich Place,Grocery Store,Pharmacy,Mobile Phone Shop,Bank,Mexican Restaurant
6,GRAND LARCENY,1.0,Pizza Place,Fast Food Restaurant,Deli / Bodega,Donut Shop,Sandwich Place,Grocery Store,Pharmacy,Mobile Phone Shop,Bank,Mexican Restaurant
7,GRAND LARCENY,1.0,Pizza Place,Fast Food Restaurant,Deli / Bodega,Donut Shop,Sandwich Place,Grocery Store,Pharmacy,Mobile Phone Shop,Bank,Mexican Restaurant
8,DANGEROUS DRUGS,1.0,Pizza Place,Donut Shop,Grocery Store,Mexican Restaurant,Deli / Bodega,Mobile Phone Shop,Sandwich Place,Spanish Restaurant,Fast Food Restaurant,Fried Chicken Joint
9,HARRASSMENT 2,1.0,Pizza Place,Deli / Bodega,Donut Shop,Grocery Store,Pharmacy,Supermarket,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Mexican Restaurant


In [67]:
pd.Series(cluster2['OFNS_DESC'].unique())

0                    DANGEROUS DRUGS
1       ASSAULT 3 & RELATED OFFENSES
2                      PETIT LARCENY
3                      GRAND LARCENY
4                      HARRASSMENT 2
5                     FELONY ASSAULT
6     CRIMINAL MISCHIEF & RELATED OF
7            MISCELLANEOUS PENAL LAW
8           OFFENSES INVOLVING FRAUD
9           VEHICLE AND TRAFFIC LAWS
10                           ROBBERY
11                          BURGLARY
12     OFF. AGNST PUB ORD SENSBLTY &
13    INTOXICATED & IMPAIRED DRIVING
14    OFFENSES AGAINST PUBLIC ADMINI
15                 DANGEROUS WEAPONS
16                           FORGERY
17               ADMINISTRATIVE CODE
18     UNAUTHORIZED USE OF A VEHICLE
19     POSSESSION OF STOLEN PROPERTY
20    GRAND LARCENY OF MOTOR VEHICLE
21                             ARSON
dtype: object

Cluster 3

In [69]:
cluster3 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[7] + list(range(24, manhattan_merged.shape[1]))]]
cluster3

,OFNS_DESC,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,OTHER OFFENSES RELATED TO THEF,2.0,Park,Gym / Fitness Center,Historic Site,Supermarket,Bus Station,Fast Food Restaurant,Mobile Phone Shop,Pharmacy,Smoke Shop,Gym
125,OTHER OFFENSES RELATED TO THEF,2.0,Park,Gym / Fitness Center,Historic Site,Supermarket,Bus Station,Fast Food Restaurant,Mobile Phone Shop,Pharmacy,Smoke Shop,Gym
139,OTHER OFFENSES RELATED TO THEF,2.0,Park,Gym / Fitness Center,Historic Site,Supermarket,Bus Station,Fast Food Restaurant,Mobile Phone Shop,Pharmacy,Smoke Shop,Gym
803,MURDER & NON-NEGL. MANSLAUGHTER,2.0,State / Provincial Park,Gym / Fitness Center,Plaza,Sandwich Place,Pizza Place,Pharmacy,Smoke Shop,Park,Fast Food Restaurant,Bus Station


In [70]:
pd.Series(cluster3['OFNS_DESC'].unique())

0     OTHER OFFENSES RELATED TO THEF
1    MURDER & NON-NEGL. MANSLAUGHTER
dtype: object

Cluster 4

In [71]:
cluster4 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[7] + list(range(24, manhattan_merged.shape[1]))]]
cluster4

,OFNS_DESC,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
119,CRIMINAL TRESPASS,3.0,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Dessert Shop,Fish Market,American Restaurant,Caribbean Restaurant,Zoo,Pharmacy
317,CRIMINAL TRESPASS,3.0,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Dessert Shop,Fish Market,American Restaurant,Caribbean Restaurant,Zoo,Pharmacy
347,CRIMINAL TRESPASS,3.0,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Dessert Shop,Fish Market,American Restaurant,Caribbean Restaurant,Zoo,Pharmacy
384,CRIMINAL TRESPASS,3.0,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Dessert Shop,Fish Market,American Restaurant,Caribbean Restaurant,Zoo,Pharmacy
536,CRIMINAL TRESPASS,3.0,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Dessert Shop,Fish Market,American Restaurant,Caribbean Restaurant,Zoo,Pharmacy
540,CRIMINAL TRESPASS,3.0,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Dessert Shop,Fish Market,American Restaurant,Caribbean Restaurant,Zoo,Pharmacy
586,CRIMINAL TRESPASS,3.0,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Dessert Shop,Fish Market,American Restaurant,Caribbean Restaurant,Zoo,Pharmacy
622,CRIMINAL TRESPASS,3.0,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Dessert Shop,Fish Market,American Restaurant,Caribbean Restaurant,Zoo,Pharmacy
667,CRIMINAL TRESPASS,3.0,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Dessert Shop,Fish Market,American Restaurant,Caribbean Restaurant,Zoo,Pharmacy
809,CRIMINAL TRESPASS,3.0,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Dessert Shop,Fish Market,American Restaurant,Caribbean Restaurant,Zoo,Pharmacy


In [73]:
pd.Series(cluster4['OFNS_DESC'].unique())

0    CRIMINAL TRESPASS
dtype: object

Cluster 5

In [74]:
cluster5 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[7] + list(range(24, manhattan_merged.shape[1]))]]
cluster5

,OFNS_DESC,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
185,THEFT-FRAUD,4.0,Supermarket,Ice Cream Shop,Gas Station,Fast Food Restaurant,Caribbean Restaurant,Bakery,Donut Shop,Fried Chicken Joint,Dessert Shop,Sandwich Place
189,THEFT-FRAUD,4.0,Supermarket,Ice Cream Shop,Gas Station,Fast Food Restaurant,Caribbean Restaurant,Bakery,Donut Shop,Fried Chicken Joint,Dessert Shop,Sandwich Place
348,THEFT-FRAUD,4.0,Supermarket,Ice Cream Shop,Gas Station,Fast Food Restaurant,Caribbean Restaurant,Bakery,Donut Shop,Fried Chicken Joint,Dessert Shop,Sandwich Place
485,THEFT-FRAUD,4.0,Supermarket,Ice Cream Shop,Gas Station,Fast Food Restaurant,Caribbean Restaurant,Bakery,Donut Shop,Fried Chicken Joint,Dessert Shop,Sandwich Place
743,THEFT-FRAUD,4.0,Supermarket,Ice Cream Shop,Gas Station,Fast Food Restaurant,Caribbean Restaurant,Bakery,Donut Shop,Fried Chicken Joint,Dessert Shop,Sandwich Place
778,THEFT-FRAUD,4.0,Supermarket,Ice Cream Shop,Gas Station,Fast Food Restaurant,Caribbean Restaurant,Bakery,Donut Shop,Fried Chicken Joint,Dessert Shop,Sandwich Place
779,THEFT-FRAUD,4.0,Supermarket,Ice Cream Shop,Gas Station,Fast Food Restaurant,Caribbean Restaurant,Bakery,Donut Shop,Fried Chicken Joint,Dessert Shop,Sandwich Place
784,THEFT-FRAUD,4.0,Supermarket,Ice Cream Shop,Gas Station,Fast Food Restaurant,Caribbean Restaurant,Bakery,Donut Shop,Fried Chicken Joint,Dessert Shop,Sandwich Place
797,THEFT-FRAUD,4.0,Supermarket,Ice Cream Shop,Gas Station,Fast Food Restaurant,Caribbean Restaurant,Bakery,Donut Shop,Fried Chicken Joint,Dessert Shop,Sandwich Place
937,THEFT-FRAUD,4.0,Supermarket,Ice Cream Shop,Gas Station,Fast Food Restaurant,Caribbean Restaurant,Bakery,Donut Shop,Fried Chicken Joint,Dessert Shop,Sandwich Place


In [75]:
pd.Series(cluster5['OFNS_DESC'].unique())

0    THEFT-FRAUD
dtype: object